In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL as image
import numpy as np
from torch.utils.data import DataLoader

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import dataloader

In [ ]:
transform = transforms.Compose(
    [
        transforms.Resize((2))
    ]
)